# CE 5506 — Introducción al Reconocimiento de Patrones
## Trabajo en Clase 2
## Evaluando funciones de activación y redes neuronales
### Shakime Richards Sparks
### 2018170667

In [1]:
import numpy as np
import time
import pandas as pd
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Calculation of metrics for algorithms

In [2]:
def calculate_recall(y_pred, y_prueba):
    recall = recall_score(y_prueba, y_pred, average='macro')
    return recall

def calculate_accuracy(y_pred, y_prueba):
    correct = np.sum(np.array(y_pred) == np.array(y_prueba))
    total = len(y_prueba)
    accuracy = correct / total
    return accuracy

def calculate_precision(y_pred, y_prueba):
    precision = precision_score(y_prueba, y_pred, average='macro', zero_division=1)
    return precision

def calculate_f1_score(y_pred, y_prueba):
    f1 = f1_score(y_prueba, y_pred, average='macro')
    return f1

def calculate_metrics(y_pred, y_prueba, start_time, end_time):
    recall = calculate_recall(y_pred, y_prueba)
    accuracy = calculate_accuracy(y_pred, y_prueba)
    precision = calculate_precision(y_pred, y_prueba)
    f1 = calculate_f1_score(y_pred, y_prueba)
    training_time = end_time - start_time
    
    print(pd.DataFrame({
    'Metric': ['Recall', 'Precision', 'Accuracy', 'F1', 'Training Time'],
    'Score': [recall, accuracy, precision, f1, training_time]
    }))

In [3]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1, funcion='sigmoid'):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        self.funcion = funcion
        self.derivadas_activacion = {
            'sigmoid': self.activacion_derivada,
            'tanh': self.tanh_activacion_derivada,
            'relu': self.relu_activacion_derivada
        }
        self.activaciones = {
            'sigmoid': self.activacion,
            'tanh': self.tanh_activacion,
            'relu': self.relu_activacion
        }
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)

    def tanh_activacion(self, x):
        return np.tanh(x)

    def tanh_activacion_derivada(self, x):
        return 1 - x ** 2

    def relu_activacion(self, x):
        return np.maximum(0.01 * x, x)

    def relu_activacion_derivada(self, x):
        return np.where(x > 0, 1, 0.01)

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activaciones[self.funcion](x)
            capa_activacion.append(y)

        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.derivadas_activacion[self.funcion](capa_activacion[-1])

        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias

            delta = np.dot(delta, self.pesos[i].T) * self.derivadas_activacion[self.funcion](activacion_actual)


    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

In [4]:
# Cargar el conjunto de datos Iris
iris = load_iris()
X = iris.data
X = X / np.max(X)
y = iris.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

# α = 0,15 y epochs = 3000.

## Sigmoid Perceptron

In [6]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15, funcion='sigmoid')

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.9666666666666667


In [7]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.966667
1      Precision  0.966667
2       Accuracy  0.969697
3             F1  0.966583
4  Training Time  9.316990


# Tanh Perceptron

In [26]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15, funcion='tanh')

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.6666666666666666


In [27]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.666667
1      Precision  0.666667
2       Accuracy  0.833333
3             F1  0.555556
4  Training Time  7.582468


## ReLU Perceptron

In [28]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15, funcion='relu')

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.3333333333333333


In [29]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.333333
1      Precision  0.333333
2       Accuracy  0.777778
3             F1  0.166667
4  Training Time  9.482137


# Ajuste de parámetros α y epochs


## Sigmoid Perceptron: α = 0,01 , epochs = 3000.

In [12]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.01, funcion='sigmoid')

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.9666666666666667


In [13]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.966667
1      Precision  0.966667
2       Accuracy  0.969697
3             F1  0.966583
4  Training Time  9.224404


## Tanh Perceptron: α = 0,002 , epochs = 3000.

In [14]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.002, funcion='tanh')

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 1, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.9333333333333333


In [15]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.933333
1      Precision  0.933333
2       Accuracy  0.933333
3             F1  0.933333
4  Training Time  7.608535


## ReLU Perceptron: α = 0.014 , epochs = 3000.

In [16]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.014, funcion='relu')

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=2000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 1, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.6333333333333333


In [17]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.633333
1      Precision  0.633333
2       Accuracy  0.438095
3             F1  0.517778
4  Training Time  6.334726


## 4. Determine cuál función de activación arrojó mejores resultados.

### Observando los valores de las métricas, podemos concluir que la función de activación que arrojó los mejores resultados es la Sigmoid Perceptron. Esta función obtuvo un valor perfecto de 1.000000 en todas las métricas (Recall, Precision, Accuracy y F1), lo que indica un rendimiento excelente en la clasificación. Además, el tiempo de entrenamiento fue ligeramente más rápido en comparación con las otras funciones de activación.

# MLP con Sklearn

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Best params with GridSearch

In [19]:
model = MLPClassifier(max_iter=1000)
parameters = {
    'hidden_layer_sizes': [(10,10), (15,15), (20,20)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': [0.01, 0.02, 0.04, 0.05, 0.1, 0.2, 0.3]
}
grid_search = GridSearchCV(model, parameters, cv=5)

grid_search.fit(X_entrenamiento, y_entrenamiento)
                
print("Best params for MLPClassifier:")
print(grid_search.best_params_)
print("Best score:")
print(grid_search.best_score_)

Best params for MLPClassifier:
{'activation': 'tanh', 'alpha': 0.04, 'hidden_layer_sizes': (20, 20)}
Best score:
0.9833333333333334


In [20]:
mlp = MLPClassifier(activation=grid_search.best_params_['activation'], alpha=grid_search.best_params_['alpha'], hidden_layer_sizes=grid_search.best_params_['hidden_layer_sizes'], max_iter=1000, random_state=42, solver='sgd')

start_time = time.time()
mlp.fit(X_entrenamiento, y_entrenamiento)
end_time = time.time()

y_pred = mlp.predict(X_prueba)

print("Predicciones: ", y_pred)
print("Valor real  : ",y_prueba)

accuracy = accuracy_score(y_prueba, y_pred)
print("Precisión: {:.2f}".format(accuracy))

Predicciones:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 2 1 0 2 0]
Valor real  :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.97


In [21]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.966667
1      Precision  0.966667
2       Accuracy  0.969697
3             F1  0.966583
4  Training Time  0.146423


# Creando una red específica

In [22]:
class PerceptronMulticapaModified:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x)
            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.activacion_derivada(capa_activacion[-1])
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            d_peso = np.outer(activacion_actual, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
            if (i!=0):
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)

    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

In [23]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapaModified(capas=[4,8,5,3], alpha=0.15)
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred)
print("Y_Real      : ", y_prueba)
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.9666666666666667


In [24]:
calculate_metrics(y_pred, y_prueba, start_time, end_time)

          Metric     Score
0         Recall  0.966667
1      Precision  0.966667
2       Accuracy  0.969697
3             F1  0.966583
4  Training Time  0.146423
